<div align="center">
<img src="https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg" width="400">
</div>

# <center> Проект: Анализ вакансий из HeadHunter
   

In [142]:
import pandas as pd
import psycopg2
# Скрыла параметры подключения в отдельном файле
from config import settings

In [143]:
# Игнорируем предупреждение pandas о SQLAlchemy
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="pandas only supports SQLAlchemy connectable")

In [144]:
connection = psycopg2.connect(**settings)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [145]:
# Текст запроса
query_3_1 = 'SELECT COUNT(*) FROM vacancies'

In [146]:
# Результат запроса
count_vacancies = pd.read_sql_query(query_3_1, connection)
total_vacancies = int(count_vacancies.iloc[0,0])
# display(count_vacancies)
print(f'Общее количество вакансий в базе - {total_vacancies}')

Общее количество вакансий в базе - 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [147]:
# Текст запроса
query_3_2 = 'SELECT COUNT(*) FROM employers'

In [148]:
# Результат запроса
count_employers = pd.read_sql_query(query_3_2, connection)
# display(count_employers)
total_employers = int(count_employers.iloc[0,0])
print(f'Общее количество работодателей в базе - {total_employers}')

Общее количество работодателей в базе - 23501


3. Посчитайте с помощью запроса количество регионов (таблица areas).

In [149]:
# Текст запроса
query_3_3 = 'SELECT COUNT(*) FROM areas'

In [150]:
# Результат запроса
count_areas = pd.read_sql_query(query_3_3, connection)
# display(count_areas)
total_areas = int(count_areas.iloc[0,0])
print(f'Общее количество регионов - {total_areas}')

Общее количество регионов - 1362


4. Посчитайте с помощью запроса количество сфер деятельности в базе (таблица industries).

In [151]:
# Текст запроса
query_3_4 = 'SELECT COUNT(*) FROM industries'

In [152]:
# Результат запроса
count_industries = pd.read_sql_query(query_3_4, connection)
# display(count_industries)
total_industries = int(count_industries.iloc[0,0])
print(f'Общее количество сфер деятельности в базе - {total_industries}')

Общее количество сфер деятельности в базе - 294


***

In [153]:
# Выводы по предварительному анализу данных
print(f'''Предварительный анализ базы данных показал:
• количество вакансий в базе - {total_vacancies},
• количество работодателей в базе - {total_employers},
• количество регионов в базе - {total_areas},
• количество сфер деятельности в базе - {total_industries}.
''')

Предварительный анализ базы данных показал:
• количество вакансий в базе - 49197,
• количество работодателей в базе - 23501,
• количество регионов в базе - 1362,
• количество сфер деятельности в базе - 294.



**Общие выводы:** база данных представляет собой крупную выборку, содержащую почти 50 тыс. вакансий от 23,5 тыс. работодателей, что обеспечивает хорошую основу для анализа. Наличие справочников по 1362 регионам и 294 отраслям предоставляет хороший потенциал для анализа, однако его глубина зависит от полноты заполнения этих данных в вакансиях.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [154]:
# Количество вакансий по регионам с сортировкой по убыванию
# Используем LEFT JOIN для включения всех регионов, даже тех где нет вакансий
query_4_1 = '''
SELECT
    a.name AS area,
    COUNT(v.id) AS cnt
FROM areas a 
LEFT JOIN vacancies v
    ON v.area_id = a.id -- Присоединяем вакансии к регионам
-- Группировка по id, для исключения одинаковых имен
GROUP BY
    a.id, a.name
-- Сортируем по убыванию количества вакансий
ORDER BY 
    cnt DESC
'''

In [155]:
# Получаем данные о распределении вакансий по регионам
area_cnt = pd.read_sql_query(query_4_1, connection)
# display(area_cnt)

# Выводим топ-5 регионов по количеству вакансий
top5_area = ", ".join(area_cnt['area'].head(5).tolist())
print(f' Регионы-лидеры по количеству размещенных вакансий - {top5_area}')

 Регионы-лидеры по количеству размещенных вакансий - Москва, Санкт-Петербург, Минск, Новосибирск, Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [156]:
# Количество вакансий, где заполнено хотя бы одно из полей с зарплатой
query_4_2 = '''
SELECT
    COUNT(id)
FROM vacancies
WHERE
    -- Учитываем вакансии с любой указанной зарплатой
    salary_from IS NOT NULL  -- "От" указана
    OR salary_to IS NOT NULL -- Или "до" указана
'''

In [157]:
# Получаем данные
salary_notnull_count = pd.read_sql_query(query_4_2, connection)
# display(salary_notnull)
salary_notnull = int(salary_notnull_count.iloc[0,0])
print(f'Количество вакансий, в которых указан уровень заработной платы - {salary_notnull}.')

Количество вакансий, в которых указан уровень заработной платы - 24073.


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [158]:
# Считаем средние только по вакансиям с указанной зарплатой
# AVG() автоматически игнорирует NULL значения
query_4_3 = '''
SELECT
    AVG(salary_from) AS avg_salary_from,
    AVG(salary_to) AS avg_salary_to
FROM vacancies
'''

In [159]:
# Результат запроса 
# Округляем результат в Pandas
salary_avg = pd.read_sql_query(query_4_3, connection).round().astype(int)
# display(salary_avg)
avg_salary_from = salary_avg.iloc[0,0]
avg_salary_to = salary_avg.iloc[0,1]
print(f'Средний уровень заработной платы в базе данных находится в диапазоне от {avg_salary_from} руб. до {avg_salary_to} руб.')

Средний уровень заработной платы в базе данных находится в диапазоне от 71065 руб. до 110537 руб.


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [160]:
# Формируем запрос с количеством вакансий по каждой форме графика и типу трудоустройства
query_4_4 = '''
SELECT
    schedule, 
    employment, 
    COUNT(id) AS count
FROM vacancies
GROUP BY
    -- Группируем по двум элементам для поиска всех сочетаний
    schedule,   --График
    employment  --Тип трудоустройства
ORDER BY 
    count DESC
'''

In [161]:
# Получаем статистику по сочетаниям графиков работы и типов трудоустройства
job_type = pd.read_sql_query(query_4_4, connection)
display(job_type.head())

# Выводим второй по популярности формат работы
second_popular = f'"{job_type.iloc[1,0]} + {job_type.iloc[1,1]}"'
print(f'На втором месте по популярности находится тип работы: "{second_popular}".')

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940


На втором месте по популярности находится тип работы: ""Удаленная работа + Полная занятость"".


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [162]:
# Запросим количество вакансий по каждому уровню опыта
query_4_5 = '''
SELECT
    experience,
    COUNT(id) AS count
FROM vacancies
GROUP BY
    experience -- Группировка по опыту
ORDER BY count -- Сортировка по количеству
'''

In [163]:
# Получаем данные о распределении вакансий по требуемому опыту
experience_df = pd.read_sql_query(query_4_5, connection)
# display(experience_df)

# Формируем строку с вариантами опыта в порядке возрастания востребованности
experience_list = ", ".join(experience_df["experience"].tolist())
print(f'Требуемый опыт работы в порядке возрастания количества вакансий: {experience_list}')

Требуемый опыт работы в порядке возрастания количества вакансий: Более 6 лет, Нет опыта, От 3 до 6 лет, От 1 года до 3 лет


***

In [164]:
# Выводы по детальному анализу вакансий
print(f'''Выводы по детальному анализу вакансий:\n
1. Топ городов с максимальным количеством размещенных вакансий: {top5_area};
2. Количество вакансий в которых указана заработная плата: {salary_notnull};
3. Уровень предлагаемых зарплат в базе данных в среднем от {avg_salary_from} руб. до {avg_salary_to} руб.;
4. На втором месте по популярности находится тип работы: {second_popular} ({job_type.iloc[1,2]} вакансии).\nНа первом месте по популярности находится тип работы: "{job_type.iloc[0,0]} + {job_type.iloc[0,1]}" ({job_type.iloc[0,2]} вакансии);
5. Наибольшее количество вакансий размещено для кандидатов с опытом работы {experience_df.iloc[-1,0].lower()} ({experience_df.iloc[-1,-1]} вакансий),\nнаименьшее -  {experience_df.iloc[0,0].lower()} опыта ({experience_df.iloc[0,1]} вакансий).
''')


Выводы по детальному анализу вакансий:

1. Топ городов с максимальным количеством размещенных вакансий: Москва, Санкт-Петербург, Минск, Новосибирск, Алматы;
2. Количество вакансий в которых указана заработная плата: 24073;
3. Уровень предлагаемых зарплат в базе данных в среднем от 71065 руб. до 110537 руб.;
4. На втором месте по популярности находится тип работы: "Удаленная работа + Полная занятость" (7802 вакансии).
На первом месте по популярности находится тип работы: "Полный день + Полная занятость" (35367 вакансии);
5. Наибольшее количество вакансий размещено для кандидатов с опытом работы от 1 года до 3 лет (26152 вакансий),
наименьшее -  более 6 лет опыта (1337 вакансий).



Анализ базы данных выявил ключевые тенденции рынка труда:

+ Вакансии сконцентрированы в крупных городах- миллионниках;
+ Около 70% всех предложений - работа в офисе на полный день, но удаленный формат прочно занимает второе место;
+ Рынок ориентирован на специалистов с опытом работы 1-3 года. Наименее востребованы опытные кандидаты (более 6 лет опыта);
+ Зарплаты сильно различаются - разброс достигает около 40% между минимальным и максимальным уровнем дохода.

Анализ подтверждает сложившуюся структуру рынка, где преобладают стандартные формы занятости и специалисты с опытом 1-3 года.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [165]:
# Формируем запрос количество вакансий от каждого работодателя
query_5_1 = '''
SELECT
    e.name,
    -- Подсчет количества вакансий каждого работодателя
    COUNT(v.id)
FROM employers e
    -- Соединяем работодателей с их вакансиями
    JOIN vacancies v ON e.id = v.employer_id
GROUP BY
    e.id, e.name -- Группируем по работодателям
ORDER BY 2 DESC
LIMIT 5          -- Берем топ-5 работодателей
'''

In [166]:
# Результат запроса
top5_employer = pd.read_sql_query(query_5_1, connection)
# display(top5_employer)

# Извлекаем работодателей с первого и пятого места
employer_1 = top5_employer.iloc[0,0]
employer_5 = top5_employer.iloc[-1,0]
print(f'На первом месте по количеству вакансий находится компания - {employer_1}')
print(f'На пятом месте по количеству вакансий находится компания - {employer_5}')

На первом месте по количеству вакансий находится компания - Яндекс
На пятом месте по количеству вакансий находится компания - Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [167]:
# Запрос для количества работодателей и вакансий для каждого региона
"""
query_5_2 = '''
SELECT
    a.name,
    COUNT(DISTINCT e.id) AS count_employer,
    COUNT(DISTINCT v.id) AS count_job
FROM areas a
    LEFT JOIN employers e ON a.id = e.area
    LEFT JOIN vacancies v ON a.id = v.area_id
GROUP BY a.id, a.name
ORDER BY count_job, count_employer DESC
'''
"""
# Предыдущий запрос хоть и короткий, но выполняется очень долго (50 сек.). 
# Попробую ускорить его с WITH оператором
query_5_2 = f'''
-- Предварительный подсчет количества работодателей по регионам
WITH cnt_employers AS (
    SELECT
        area, COUNT(*) AS count
    FROM employers
    GROUP BY area
),

-- Предварительный подсчет количества вакансий по регионам
cnt_vacancy AS (
    SELECT
        area_id, COUNT(*) AS count
    FROM vacancies
    GROUP BY area_id
)

-- Финальный запрос: объединяем регионы с предварительно посчитанной статистикой
SELECT 
    a.name AS region_name,
    -- Заменяем NULL на 0 для регионов без работодателей
    COALESCE(e.count, 0) AS count_employer,
    -- Заменяем NULL на 0 для регионов без вакансий
    COALESCE(v.count, 0) AS count_job
FROM areas a
-- Статистика по работодателям
LEFT JOIN cnt_employers e ON a.id = e.area
-- Статистика по вакансиям
LEFT JOIN cnt_vacancy v ON a.id = v.area_id
ORDER BY 
    count_job,          --сначала регионы без вакансий
    count_employer DESC -- далее по убыванию кол-ва работодателей
'''

In [168]:
# Результат запроса
employer_in_area = pd.read_sql_query(query_5_2, connection)
# display(employer_in_area)

# Первая строка результата будет содержать регион без вакансий с максимальным количеством работодателей
print(f'Среди регионов без вакансий лидирует по количеству работодателей: {employer_in_area.iloc[0,0]}')

Среди регионов без вакансий лидирует по количеству работодателей: Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [169]:
# Подсчет количества регионов, в которых каждый работодатель публикует вакансии
query_5_3 = '''
SELECT
    e.name,
    -- Уникальные регионы для каждого работодателя
    COUNT(DISTINCT v.area_id) AS count_area
FROM employers e
    -- LEFT JOIN чтобы включить работодателей без вакансий
    LEFT JOIN vacancies v ON e.id = v.employer_id
GROUP BY e.id, e.name
-- Сортировка по убыванию количества регионов
ORDER BY count_area DESC
'''

In [170]:
# Результат запроса
cnt_emp_area = pd.read_sql_query(query_5_3, connection)
# Топ-5 работодателей по охвату регионов
display(cnt_emp_area.head())

# Выводим работодателя с максимальным географическим охватом
max_regions = cnt_emp_area.iloc[0,1]
top_employer = cnt_emp_area.iloc[0,0]
print(f'Максимальное количество регионов в которых публикуются вакансии \
от одного работодателя - {max_regions}, работодатель - "{top_employer}"')

,name,count_area
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


Максимальное количество регионов в которых публикуются вакансии от одного работодателя - 181, работодатель - "Яндекс"


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [171]:
# Количество работодателей с NULL сферами деятельности
query_5_4 = '''
SELECT
    COUNT(*)
FROM employers e
    LEFT JOIN employers_industries ei ON e.id = ei.employer_id
WHERE
    -- Работодатели без указания сферы деятельности
    industry_id IS NULL 
'''

In [172]:
# Результат запроса
null_industries_count = pd.read_sql_query(query_5_4, connection)
# display(null_industries_count)

null_industries = int(null_industries_count.iloc[0,0])
print(f'{null_industries} работодателей в базе данных не указали сферу деятельности.')

8419 работодателей в базе данных не указали сферу деятельности.


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [173]:
# Запросим компании и их сферы деятельности
query_5_5 = f'''
SELECT
    e.name
FROM employers e
    -- Соединяем с таблицей связей со сферами
    JOIN employers_industries ei ON e.id = ei.employer_id
GROUP BY
    e.id,
    e.name -- Группируем по работодателям
HAVING
    -- Фильтруем только компании с 4 уникальными сферами
    COUNT(DISTINCT ei.industry_id) = 4
ORDER BY name -- сортируем по алфавиту
OFFSET 2   -- Пропускаем первые две записи (0-я и 1-я)
LIMIT 1    -- Берем только одну запись (третью по порядку)
'''

In [174]:
# Результат запроса
cnt_industries = pd.read_sql_query(query_5_5, connection)

# Выводим компанию, занимающую третье место в алфавитном списке
company_name = cnt_industries.iloc[0,0]
print(f'Среди компаний с 4 сферами деятельности третье место в алфавитном порядке занимает "{company_name}"')

Среди компаний с 4 сферами деятельности третье место в алфавитном порядке занимает "2ГИС"


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [175]:
# Находим количество компаний с определенной сферой деятельности
query_5_6 = f'''

SELECT
    COUNT(*)
FROM employers e
    -- Связь работодателей со сферами
    JOIN employers_industries ei ON e.id = ei.employer_id 
    -- Получаем названия сфер деятельности
    JOIN industries i ON i.id = ei.industry_id
WHERE
    -- Фильтруем по конкретной сфере
    i.name = 'Разработка программного обеспечения'
'''

In [176]:
# Результат запроса
it_industry_count = pd.read_sql_query(query_5_6, connection)
# display(it_industry_count)

it_industry = int(it_industry_count.iloc[0,0])
print(f'"Разработка программного обеспечения" в качестве сферы деятельности указана у {it_industry} работодателей.')

"Разработка программного обеспечения" в качестве сферы деятельности указана у 3553 работодателей.


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [177]:
# Код для получения списка городов-милионников
import requests
from bs4 import BeautifulSoup
from pprint import pprint

url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
# GET-запрос к странице Википедии
response = requests.get(url, headers={'User-Agent': 'Chrome/141.0.0.0'})
# Создаем объект BeautifulSoup
page = BeautifulSoup(response.text, 'html.parser')
# Поиск таблицы с городами и извлечение всех ссылок (названий городов)
links = page.find('table', 'standard sortable').find_all('a')
# Формирование списка городов-миллионников (пропускаем первые 5 служебных ссылок)
million_cities = tuple(link.text for link in links[5:])
# pprint(million_cities)

In [178]:
# Запрос количества вакансий компании Яндекс, в городах миллиониках
query_5_7 = f'''
-- WITH для подсчета вакансий Яндекса по городам-миллионникам
WITH yandex_area_cnt AS(
    SELECT
        a.name AS city,
        COUNT(*)
    FROM vacancies v
        -- Связываем вакансии с регионами
        JOIN areas a ON a.id = v.area_id
        -- Связываем вакансии с работодателями
        JOIN employers e ON e.id = v.employer_id 
    WHERE
        e.name = 'Яндекс'              -- Фильтруем по компании Яндекс
        AND a.name IN {million_cities} -- Только города-миллионники
    -- группируем по городам для подсчета
    GROUP BY
        a.id,
        a.name
)
-- Основные данные по городам + строка с суммой    
SELECT * FROM yandex_area_cnt
UNION
SELECT
    'Total',                           -- Строка с общим итогом
    SUM(count)
FROM yandex_area_cnt
ORDER BY count
'''

In [179]:
# Результат запроса
yandex_area_cnt = pd.read_sql_query(query_5_7, connection)
# Отображаем таблицу
display(yandex_area_cnt)

# Сколько строк получилось в вашей выборке?
print(f'Выборка содержит {yandex_area_cnt.shape[0]} строк')

# Какой результат получился в строке Total?
total = int(yandex_area_cnt[yandex_area_cnt['city'] == 'Total'].iloc[0,1])
print(f'Всего в городах-миллионниках компанией Яндекс размещено {total} вакансий')

,city,count
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


Выборка содержит 17 строк
Всего в городах-миллионниках компанией Яндекс размещено 485 вакансий


***

In [180]:
# Выводы по анализу работодателей
print(f'''Выводы по анализу работодателей:
\n1. Топ 5 работодателей в базе данных по количеству размещенных вакансий: {', '.join(top5_employer['name'])}.
Вывод: IT-ти и банковские отрасли являются лидирующими на рынке труда;
\n2. Регион с максимальным количеством работодателей, где отсутствуют размещенные вакансии - {employer_in_area.iloc[0,0]}.
Вывод: работодатели обычно указывают конкретный город в вакансиях и редко ищут кандидатов по всей стране;
\n3. Максимальное количество регионов в которых публикуются вакансии от одного работодателя - {cnt_emp_area.iloc[0,1]}, работодатель - "{cnt_emp_area.iloc[0,0]}". 
Вывод: для IT-индустрии географическое расположение не является существенным фактором;
\n4. {null_industries} работодателей в базе данных ({null_industries/total_employers*100:.1f}% от общего числа) не указали сферу деятельности. 
Тогда как, например, компания "{company_name}" имеет 4 сферы деятельности;
Вывод: большое количество работодателей не заполняют данные, тогда как некоторые указывают несколько сфер
\n6. Сферу "Разработка программного обеспечения" указали {it_industry} работодателей ({it_industry/total_employers*100:.1f}% от общего числа).
Вывод: высокая востребованность IT-специальностей на рынке труда
\n7. В России {len(million_cities)} городов-миллионников. Компанией Яндекс (лидер по количеству размещенных вакансий), \nлишь треть вакансий ({total}) размещены в городах миллиониках.
Вывод: IT-специальности имеют широкий территориальный охват за пределами крупных городов
''')

Выводы по анализу работодателей:

1. Топ 5 работодателей в базе данных по количеству размещенных вакансий: Яндекс, Ростелеком, Тинькофф, СБЕР, Газпром нефть.
Вывод: IT-ти и банковские отрасли являются лидирующими на рынке труда;

2. Регион с максимальным количеством работодателей, где отсутствуют размещенные вакансии - Россия.
Вывод: работодатели обычно указывают конкретный город в вакансиях и редко ищут кандидатов по всей стране;

3. Максимальное количество регионов в которых публикуются вакансии от одного работодателя - 181, работодатель - "Яндекс". 
Вывод: для IT-индустрии географическое расположение не является существенным фактором;

4. 8419 работодателей в базе данных (35.8% от общего числа) не указали сферу деятельности. 
Тогда как, например, компания "2ГИС" имеет 4 сферы деятельности;
Вывод: большое количество работодателей не заполняют данные, тогда как некоторые указывают несколько сфер

6. Сферу "Разработка программного обеспечения" указали 3553 работодателей (15.1% от общег

**Общий вывод:** Анализ работодателей выявил лидерство IT и банковского сектора. При этом IT-компании демонстрируют наибольший географический охват, активно нанимая за пределами крупных городов. Однако стоит учитывать в дальнейшем анализе, что качество данных ограничено - каждый третий работодатель не указал сферу деятельности. В целом рынок характеризуется концентрацией вакансий у крупных игроков при широком региональном распределении.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [181]:
# Количество вакансий имеющих отношение к данным
query_6_1 = f'''
SELECT
    COUNT(*)
FROM vacancies
WHERE
    name ILIKE '%data%'    -- Поиск слова 'data' в любом регистре
    OR name ILIKE '%данн%' -- Поиск 'данн' в любом регистре
'''

In [182]:
# Результат запроса
data_vacancies_df = pd.read_sql_query(query_6_1, connection)
# display(data_vacancies_df)

data_vacancies = int(data_vacancies_df.iloc[0,0])
print(f'Количество вакансий, связанных с данными: {data_vacancies}')

Количество вакансий, связанных с данными: 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [183]:
# Количество подходящих вакансий для начинающего DS
# Формируем условия для поиска Data Science вакансий
ds_job_list = [
    'data scientist', 
    'data science', 
    'исследователь данных',
    'machine learning', 
    'машинн%обучен'
]
ds_job_list = ' OR '.join([f"name ILIKE '%{elem}%'" for elem in ds_job_list])

# Базовый запрос для выделения всех DS-вакансий (будет использоваться в последующих заданиях)
ds_vacancy = f'''
    SELECT *
    FROM vacancies
    WHERE {ds_job_list}                -- Вакансии из списка ds_job_list
        -- Добавляем поиск по ML вакансиям, исключая HTML в любом регистре
        OR (name LIKE '%ML%' AND name NOT ILIKE '%HTML%')
'''
# Основной запрос: поиск Junior-позиций среди DS-вакансий
query_6_2 = f'''
-- Используем WITH с ранее определенными DS-вакансиям
WITH ds_vacancy AS ({ds_vacancy})
SELECT
    COUNT(*)
FROM ds_vacancy
WHERE
    name ILIKE '%junior%'              -- В названии есть слово junior
    OR experience ILIKE '%Нет опыта%'  -- Требуемый опыт: нет опыта
    OR employment ILIKE '%Стажировка%' -- Тип трудоустройства: стажировка
'''

In [184]:
# Результат запроса
junior_ds_vacancies_df = pd.read_sql_query(query_6_2, connection)
# display(junior_ds_vacancies)

junior_ds_vacancies = int(junior_ds_vacancies_df.iloc[0,0])
print(f'Количество вакансий для начинающего дата-сайентиста: {junior_ds_vacancies}')

Количество вакансий для начинающего дата-сайентиста: 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [185]:
# Текст запроса для подсчета DS-вакансий с навыками SQL/PostgreSQL
query_6_3 = f'''
WITH ds_vacancy AS ({ds_vacancy})    -- Используем наше определение DS-вакансий
SELECT
    COUNT(*)
FROM ds_vacancy
WHERE
    key_skills ILIKE '%SQL%'         -- Поиск SQL в навыках (без учета регистра)
    OR key_skills ILIKE '%postgres%' -- Поиск PostgreSQL в навыках (без учета регистра)
'''

# Альтернативный запрос - для проверки расхождений с эталонным ответом
# В базовом запросе используется LIKE вместо ILIKE для фильтрации ML-вакансий
query_6_3_1 = f'''
WITH ds_vacancy AS (
    SELECT *
    FROM vacancies
    WHERE {ds_job_list}
        -- Исправление: ILIKE вместо LIKE
        OR (name ILIKE '%ML%' AND name NOT ILIKE '%HTML%')
)
SELECT
    COUNT(*)
FROM ds_vacancy
WHERE
    key_skills ILIKE '%SQL%'
    OR key_skills ILIKE '%postgres%'
'''

In [186]:
# Выполнение основного запроса
sql_postgres_skills_df = pd.read_sql_query(query_6_3, connection)
# display(sql_postgres_skills)
sql_postgres_skills = int(sql_postgres_skills_df.iloc[0,0])

# Выполнение альтернативного запроса (для сравнения с эталоном)
for_answer_df = pd.read_sql_query(query_6_3_1, connection)
# display(for_answer_df)
for_answer = int(for_answer_df.iloc[0,0])

# Анализ результатов
difference = for_answer - sql_postgres_skills
print(f'''Результаты подсчета DS-вакансий с навыками SQL/PostgreSQL:
• Мой подсчет: {sql_postgres_skills} вакансий
• Эталонный подсчет: {for_answer} вакансий
• Расхождение: {difference} вакансий''')

Результаты подсчета DS-вакансий с навыками SQL/PostgreSQL:
• Мой подсчет: 201 вакансий
• Эталонный подсчет: 229 вакансий
• Расхождение: 28 вакансий


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [187]:
# Текст запроса для подсчета DS-вакансий с навыком Python
query_6_4 = f'''
WITH ds_vacancy AS ({ds_vacancy})
SELECT
    COUNT(*)
FROM ds_vacancy
WHERE
    -- поиск Python в навыках (без учета регистра)
    key_skills ILIKE '%Python%'
'''

In [188]:
# Результат запроса
python_skill_df = pd.read_sql_query(query_6_4, connection)
# display(python_skill_df)
python_skill = int(python_skill_df.iloc[0,0])
print(f'Python указан в качестве ключевого навыка в {python_skill} вакансии.')

Python указан в качестве ключевого навыка в 351 вакансии.


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [189]:
# Сравнение двух подходов к подсчету среднего количества навыков - Python vs SQL
# (данное сравнение не требовалось в задании, выполнено из интереса)

# Подход 1: Получение данных и расчет в Python
# Запрос для получения всех ненулевых навыков из DS-вакансий
query_6_5_1 = f'''
WITH ds_vacancy AS ({ds_vacancy})
SELECT
    key_skills
FROM ds_vacancy
WHERE
    -- исключаем NULL для корректной обработки в Python
    key_skills IS NOT NULL
'''

# Подход 2: Полный расчет в SQL
# Используем разницу длин строк для подсчета количества навыков (разделитель - '\t')
query_6_5_2 = f'''
WITH ds_vacancy AS ({ds_vacancy})
SELECT 
    AVG(
        -- Подсчет количества навыков через количество разделителей:
        -- (общая длина - длина без разделителей) + 1
        LENGTH(key_skills) - LENGTH(REPLACE(key_skills, '\t', '')) + 1 
    ) AS avg_skills_sql
FROM ds_vacancy
-- В SQL AVG() автоматически игнорирует NULL значения
'''


In [190]:
# Выполнение запросов
skills_for_python_df = pd.read_sql_query(query_6_5_1, connection)
sql_calculation_df = pd.read_sql_query(query_6_5_2, connection)

# Расчет среднего количества навыков в Python
skills_for_python_df['skills_count'] = skills_for_python_df['key_skills'].apply(lambda x: len(x.split('\t')))
avg_skills_python = skills_for_python_df['skills_count'].mean().round(2)

# Расчет средствами SQL
avg_skills_sql = float(sql_calculation_df.iloc[0,0])

# Сравнение результатов и вывод ответа
print(f'''Результаты подсчета среднего количества ключевых навыков в DS-вакансиях:
• SQL-подсчет: {avg_skills_sql:.2f} навыков
• Python-подсчет: {avg_skills_python} навыков''')

Результаты подсчета среднего количества ключевых навыков в DS-вакансиях:
• SQL-подсчет: 6.41 навыков
• Python-подсчет: 6.41 навыков


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [191]:
# Расчет средней зарплаты для Data Science по уровням опыта
query_6_6 = f'''
WITH ds_vacancy AS ({ds_vacancy})
SELECT
    experience,
    AVG(
        COALESCE(
            -- если обе зарплаты указаны - берем среднее
            (salary_from + salary_to) / 2,
            salary_from,     -- если указана только from
            salary_to        -- если указана только to
            )
        ) AS avg_salary
FROM ds_vacancy
WHERE
    salary_from IS NOT NULL 
    OR salary_to IS NOT NULL -- фильтруем вакансии с указанной зарплатой
GROUP BY experience
'''

In [192]:
# Результат запроса
exp_avg_salary = pd.read_sql_query(query_6_6, connection)
# Округляем зарплаты до целых чисел
exp_avg_salary['avg_salary'] = exp_avg_salary['avg_salary'].round().astype(int)
# Таблица с результатами
display(exp_avg_salary)

# Извлекаем зарплату для специалистов с опытом 3-6 лет
salary_3_6_years = exp_avg_salary[
    exp_avg_salary['experience'] == 'От 3 до 6 лет'
]['avg_salary'].values[0]

print(f'Дата-сайентист с опытом работы от 3 до 6 лет может претендовать в среднем на {salary_3_6_years} рублей.')

,experience,avg_salary
0,От 3 до 6 лет,243115
1,От 1 года до 3 лет,139675
2,Нет опыта,74643


Дата-сайентист с опытом работы от 3 до 6 лет может претендовать в среднем на 243115 рублей.


***

In [193]:
print(f'''
Выводы по предметному анализу данных:
      
1. Доля вакансий, связанных с данными, составляет {data_vacancies/total_vacancies*100:.1f}% от общего числа, 
что указывает на то, что профессии, связанные с данными, остаются специализированными и требуют высокой квалификации.

2. Для начинающих дата-сайентистов (Junior) доступно всего {junior_ds_vacancies} вакансия. 
Это очень ограниченное количество точек входа в профессию для новичков, что указывает на то, 
что большинство компаний ищут уже подготовленных специалистов.

3. Ключевые навыки. Python доминирует как основной инструмент (требуется в {python_skill} вакансиях). 
SQL также является критически важным навыком (указан в {sql_postgres_skills} вакансиях). 
Данные показатели подтверждают, что ключевые навыки в DS строятся на программировании и работе с базами данных.
В среднем в вакансиях для DS указывается {avg_skills_python} ключевых навыков, что отражает высокие требования к специалистам DS.

4. Зарплатная вилка ({exp_avg_salary.iloc[2,1]}→{exp_avg_salary.iloc[1,1]}→{exp_avg_salary.iloc[0,1]} руб.) показывает сильную зависимость дохода от опыта работы. 
Разрыв почти в 3,5 раза между начинающими и опытными специалистами.
''')


Выводы по предметному анализу данных:

1. Доля вакансий, связанных с данными, составляет 3.6% от общего числа, 
что указывает на то, что профессии, связанные с данными, остаются специализированными и требуют высокой квалификации.

2. Для начинающих дата-сайентистов (Junior) доступно всего 51 вакансия. 
Это очень ограниченное количество точек входа в профессию для новичков, что указывает на то, 
что большинство компаний ищут уже подготовленных специалистов.

3. Ключевые навыки. Python доминирует как основной инструмент (требуется в 351 вакансиях). 
SQL также является критически важным навыком (указан в 201 вакансиях). 
Данные показатели подтверждают, что ключевые навыки в DS строятся на программировании и работе с базами данных.
В среднем в вакансиях для DS указывается 6.41 ключевых навыков, что отражает высокие требования к специалистам DS.

4. Зарплатная вилка (74643→139675→243115 руб.) показывает сильную зависимость дохода от опыта работы. 
Разрыв почти в 3,5 раза между начинающими 

**Основные выводы исследования рынка вакансий Data Science:**

- Рынок данных небольшой и требовательный. Вакансии, связанные с данными, занимают лишь около 3.6% от общего числа. Для работы в Data Science нужно много навыков — в среднем 6–7 на одну позицию. Новичкам сложно попасть в отрасль: всего 51 вакансия уровня junior.
- Главные инструменты Data Science — Python и SQL. Python используется чаще всего (351 вакансия), SQL — второй по важности (201 вакансия). То есть без уверенного владения программированием и работой с данными в профессию не попасть.
- Зарплаты сильно растут с опытом. Junior-специалисты получают около 74 000 ₽, с опытом 1–3 года — примерно 140 000 ₽, а после 3 лет — около 243 000 ₽. Разница между новичками и опытными специалистами — более чем в 3 раза.

Вывод: Data Science остается элитной профессией с высоким порогом входа, значительными требованиями к навыкам, 
но с перспективной зарплатной прогрессией для тех, кто готов развиваться в данной отрасли.

# Общий вывод по проекту и дополнительные исследования

## Дополнительные исследования

### 1. Анализ расхождений в задании 6.3

Возвращаясь к заданию 6.3 настоящей работы - "Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?" 
Хочу понять, почему мой ответ (201) не сходится с эталонным (229). Для этого задам запрос, где будут указаны "спорные" 28 вакансий.

In [194]:
# Запрос для выявления вакансий, которые попадают в выборку при ILIKE, но не попадают при LIKE
# Это поможет найти источник расхождения в 28 вакансий
query_ml_diff_sql = '''
SELECT name
FROM vacancies
WHERE
    -- условие из "правильного" запроса
        (name ILIKE '%ML%' AND name NOT ILIKE '%HTML%')
    -- исключаем вакансии, которые нашел бы LIKE
    AND (name NOT LIKE '%ML%' AND name NOT ILIKE '%HTML%')
    -- только вакансии с SQL/PostgreSQL
    AND (key_skills ILIKE '%SQL%' OR key_skills ILIKE '%postgres%')
'''
# Выполняем запрос и анализируем результаты
diff_sql_vacancies = pd.read_sql_query(query_ml_diff_sql, connection)
print(f"Вакансии с SQL которые есть только при вызове ILIKE 'ML': {len(diff_sql_vacancies)}")
display(diff_sql_vacancies)

Вакансии с SQL которые есть только при вызове ILIKE 'ML': 28


,name
0,QA Automation TeamLead
1,PHP Senior/TeamLead
2,TeamLead группы системного администрирования
3,Senior Developer (teamlead)
4,Teamlead Java
5,TeamLead
6,Руководитель группы разработки (TeamLead Java)
7,TeamLead Analyst BI (AI Driving Data)
8,Teamlead отдела системной аналитики
9,Fullstack Teamlead (Javascript vue.js + node.js)


Как мы видим, все эти вакансии не связаны с Data Science, а связаны с поиском **TeaMLead**. Это ложное срабатывание фильтра по "ML". **Вывод:** мой ответ - 201 вакансия, остается верным, так как эталонный подсчет включает не нужные для исследования вакансии.

### 2. Количество Data Science вакансий и их доля на рынке всех вакансий

В процессе работы мы работали с данными о вакансиях связанных с DS, но так и не получили данные, сколько всего связанных с DS вакансий размещено и какую долю от общего числа вакансий они составляют

In [195]:
# Подсчет общего количества DS-вакансий и их доли на рынке
ds_vacancy_count = f'''
WITH ds_vacancy AS ({ds_vacancy})
SELECT COUNT(*) 
FROM ds_vacancy
'''
# Выполняем запрос и извлекаем результат
ds_vacancy_total = int(pd.read_sql_query(ds_vacancy_count, connection).iloc[0,0])
# Рассчитываем долю DS-вакансий от общего числа
ds_share = round(ds_vacancy_total/total_vacancies*100, 1)

print(f'Всего DS-вакансий: {ds_vacancy_total}')
print(f'Их доля в общем числе вакансий: {ds_share}%')

Всего DS-вакансий: 480
Их доля в общем числе вакансий: 1.0%


**Вывод:**
Направление Data Science занимает ограниченную, но значимую долю на рынке труда. При общем количестве 49 197 вакансий, к сфере Data Science относятся 480 позиций, что составляет порядка 1% от совокупного предложения.

Данные подтверждают ранее выявленные закономерности: Data Science — высокоспециализированная область с повышенным порогом входа, направление не является массовым и требует глубоких профессиональных навыков.

### 3. Сравним DS с другими IT вакансиями. Какая доля DS среди всех IT-вакансий?

In [196]:
# Сравнение доли DS среди IT-вакансий
# Сначала определим id сферы деятельности - "Разработка программного обеспечения"
id_it_industry_query = '''
SELECT id
FROM industries
WHERE name = 'Разработка программного обеспечения'
'''
# Кладем id в переменную
id_it_industry = pd.read_sql_query(id_it_industry_query, connection).iloc[0,0]

# Найдем общее количество IT-вакансий (вакансии от работодателя с сферой деятельности - разработка ПО)
it_vacancies_query = f'''
SELECT
    COUNT(*)
FROM vacancies v
    -- Присоединяем работодателей к вакансиям
    JOIN employers e ON v.employer_id = e.id
    --Присоединяем данные об индустриях
    JOIN employers_industries ei ON e.id = ei.employer_id
WHERE
    -- где ID "Разработка программного обеспечения"
    ei.industry_id = '{id_it_industry}'
'''

# Выполняем запросы
it_vacancies_total = int(pd.read_sql_query(it_vacancies_query, connection).iloc[0,0])
# Рассчитываем долю
ds_share_in_it = round(ds_vacancy_total / it_vacancies_total * 100, 1)

print(f'''СРАВНЕНИЕ DS С IT-СЕКТОРОМ:
• Всего IT-вакансий (разработка ПО): {it_vacancies_total}
• Всего DS-вакансий: {ds_vacancy_total}
• Доля DS среди IT-вакансий: {ds_share_in_it}%''')

СРАВНЕНИЕ DS С IT-СЕКТОРОМ:
• Всего IT-вакансий (разработка ПО): 12499
• Всего DS-вакансий: 480
• Доля DS среди IT-вакансий: 3.8%


**Вывод:** Data Science занимает небольшую долю в сегменте разработки программного обеспечения — около 3.8%. Примерно такая же доля сохраняется и среди всех вакансий, связанных с данными, по отношению к общему рынку труда. Это подтверждает, что направление остаётся нишевым и специализированным, с ограниченным количеством позиций и высоким уровнем требований к квалификации специалистов.

### 4. География зарплат в Data Science

4.1. Средняя зарплата Data Science по городам-миллионникам

In [197]:
# Добавлю библиотеку для построения графиков
import plotly.express as px

# Сравним уровень зарплат DS по городам-миллионникам
area_salary_query = f'''
WITH ds_vacancy AS ({ds_vacancy})
SELECT 
    a.name AS city,
    COUNT(*) AS vacancies_count,
    -- Округляю значение сразу в SQL:
    -- беру среднее между salary_from и salary_to, если оба указаны;
    -- иначе использую ненулевое поле.
    ROUND(
        AVG(
            COALESCE(
                (salary_from + salary_to)/2, 
                salary_from, 
                salary_to
            )
        )
    ) AS avg_salary                         -- Средняя заработная плата
FROM ds_vacancy v
JOIN areas a ON v.area_id = a.id            -- Присоединяю таблицу регионов
WHERE
    a.name IN {million_cities}              -- Фильтрую по городам-миллионникам
    AND (salary_from IS NOT NULL 
         OR salary_to IS NOT NULL)          -- Только вакансии, где указана зарплата
GROUP BY a.id, a.name                       -- Группирую результаты по городу
ORDER BY avg_salary DESC                    -- Сортирую по убыванию зарплаты
'''
# Получаю данные
area_salary_df = pd.read_sql_query(area_salary_query, connection)

In [198]:
# Создаем интерактивный столбчатый график
fig_1 = px.bar(
    area_salary_df,
    x='city',
    y='avg_salary',
    title='Средняя зарплата Data Science по городам-миллионникам',
    labels={'city': 'Город', 'avg_salary': 'Средняя зарплата, руб.'},
    color='avg_salary',  # Раскрашиваем столбцы по значению зарплаты
    color_continuous_scale='reds',
    text='avg_salary'  # Показываем значения на столбцах
)

# Настраиваем оформление
fig_1.update_layout(
    xaxis_tickangle=-45,  # Наклон подписей городов
    #showlegend=False,     # Скрываем легенду для цветовой шкалы
    plot_bgcolor='white', # Белый фон
    width=800,
    height=500,
    coloraxis_colorbar=dict(title='')  # Убираем подпись у цветовой шкалы
)

# Форматируем подписи на столбцах
fig_1.update_traces(
    texttemplate='%{text:.0f} руб.',
    textposition='outside',
    hovertemplate='<b>%{x}</b><br>Средняя зарплата: %{y:.0f} руб.<extra></extra>'
)

# Настраиваем оси, добавляем сетку
fig_1.update_yaxes(
    showgrid=True, 
    gridwidth=1, 
    gridcolor='lightgray',
    showticklabels=False  # Убираю числовые подписи по оси Y
)

# fig_1.show()
# fig_1.write_html("data/html/fig_1.html")

<img src = 'data/image/fig_1.png'>

Ознакомится, с интерактивным графиком можно по [ссылке](https://htmlpreview.github.io/?https://github.com/emozdir/project_2/blob/master/data/html/fig_1.html)

**Выводы**:
Анализ данных по 16 городам-миллионникам показал, что информация о заработных платах Data Science-специалистов представлена только в 8 из них. В остальных городах либо отсутствуют вакансии по направлению Data Science, либо работодатели не указали уровень оплаты труда.
Наибольшие средние зарплаты зафиксированы в Москве (свыше 200 тыс. руб.), что отражает концентрацию крупнейших ИТ-компаний и исследовательских центров в столичном регионе. Санкт-Петербург и Новосибирск занимают следующие позиции (170–175 тыс. руб.). В остальных городах, где данные доступны (Екатеринбург, Уфа, Пермь, Ростов-на-Дону, Казань), уровень дохода существенно ниже — от 50 до 125 тыс. руб.
Зарплаты в Москве превышают региональные в среднем в четыре раза, что подтверждает неравномерность распределения вакансий с высокими зарплатами и их концентрацию в крупнейших технологических центрах страны.

4.2. Средняя зарплата Data Science по всем городам

In [199]:
# Сравним уровень зарплат DS по всем городам
# Запрос аналогичен предыдущему, убираю только фильтр по городам миллионникам
all_cities_salary_query = f'''
WITH ds_vacancy AS ({ds_vacancy})
SELECT 
    a.name AS city,
    COUNT(*) AS vacancies_count,
    ROUND(
        AVG(
            COALESCE(
                (salary_from + salary_to)/2, 
                salary_from, 
                salary_to
            )
        )
    ) AS avg_salary
FROM ds_vacancy v
JOIN areas a ON v.area_id = a.id
WHERE
    (salary_from IS NOT NULL 
     OR salary_to IS NOT NULL)
GROUP BY a.id, a.name
ORDER BY avg_salary DESC
'''

In [200]:
# Получаю данные
all_cities_salary_df = pd.read_sql_query(all_cities_salary_query, connection)

# Создаем интерактивный столбчатый график для всех городов
fig_1_1 = px.bar(
    all_cities_salary_df,
    x='city',
    y='avg_salary',
    title='Средняя зарплата Data Science по всем городам',
    labels={'city': 'Город', 'avg_salary': 'Средняя зарплата, руб.'},
    color='avg_salary',
    color_continuous_scale='reds',
    text='avg_salary'
)

# Настраиваю оформление
fig_1_1.update_layout(
    xaxis_tickangle=-45,
    plot_bgcolor='white',
    width=1000,  # Увеличиваем ширину для большего количества городов
    height=600,
    coloraxis_colorbar=dict(title='')
)

# Форматирую подписи на столбцах
fig_1_1.update_traces(
    texttemplate='%{text:.0f} руб.',
    # Подписи ВНУТРИ блоков
    textposition='inside',
    textangle=-90,
    insidetextanchor='middle',
    hovertemplate=(
    '<b>%{x}</b><br>Средняя зарплата: %{y:.0f} руб.<br>'
    'Вакансий: %{customdata[0]}<extra></extra>')
)

# Настраиваем оси
fig_1_1.update_yaxes(
    showgrid=True, 
    gridwidth=1, 
    gridcolor='lightgray',
    showticklabels=False
)

# fig_1_1.show()
# fig_1_1.write_html("data/html/fig_1.html")

<img src = 'data/image/fig_1_1.png'>

Ознакомится, с интерактивным графиком можно по [ссылке](https://htmlpreview.github.io/?https://github.com/emozdir/project_2/blob/master/data/html/fig_1_1.html)

**Сравнительный анализ зарплат Data Science по городам России и зарубежья**

Анализ всех городов выявил новые тенденции в оплате труда Data Science-специалистов:

1. Международные рынки конкурируют с Москвой. Зарубежные локации (Кипр, Армения, Сербия, Турция) предлагают зарплаты на уровне или выше московских (233-300 тыс. руб.)

2. Москва теряет монополию на высокие зарплаты - 207 тыс. руб. в столице против 233-300 тыс. руб. в зарубежных странах

3. Российские регионы показывают значительный разброс:
   - Высокий уровень: Белгород (200 тыс. руб.) неожиданно обгоняет Санкт-Петербург
   - Средний уровень: СПб, Новосибирск, Рязань, Королев (130-173 тыс. руб.)
   - Базовый уровень: Екатеринбург, Уфа и др. (85-124 тыс. руб.)

4. Высокие зарплаты появляются в региональных городах России вне городов-миллионников (Белгород, Рязань)

**Ключевой вывод:** Рынок Data Science становится более распространенным — высокие зарплаты доступны не только в крупных городах России, но и в зарубежных странах и отдельных российских регионах, что создает новые возможности для карьерного роста вне городов-миллионников.

### 5. Распределение требуемых навыков по уровням опыта работы

In [61]:
# Формируем запрос (получаем таблицу с опытом и всеми навыками)
skills_exp_query = f'''
WITH ds_vacancy AS ({ds_vacancy})
SELECT
    experience, 
    key_skills AS skill
FROM ds_vacancy
WHERE
    key_skills IS NOT NULL 
    AND experience IS NOT NULL
'''
# Формируем DataFrame
skills_exp_df = pd.read_sql_query(skills_exp_query, connection)

# Разбиваем навыки на списки
skills_exp_df['skill'] = skills_exp_df['skill'].str.split('\t')

# Преобразовываем каждый элемент списка в строку
skills_exp_df = skills_exp_df.explode('skill')

# Создаем группы, где в каждой группе одинаковые пары (опыт, навык)
skills_exp_df = (skills_exp_df.groupby(['experience', 'skill'])
                 # Подсчитываем количество строк в каждой группе
                 .size()
                 # Сбрасываем индекс, добавляем столбец с количеством
                 .reset_index(name='skill_count')
                 # Сортируем по опыту, потом по количеству в порядке убывания
                 .sort_values(['experience', 'skill_count'], ascending=[True, False]))

# Берем топ-10 для каждого уровня опыта
# Группируем по опыту, и выводим первые 10 значений
top_skills_exp_df = (skills_exp_df.groupby('experience')
                     .head(10)
                     .reset_index(drop=True))

In [62]:
# Строим график
# Создаем матрицу, навык/опыт, количество в качестве значений, если не указанно ставим 0
pivot_df = top_skills_exp_df.pivot(index='skill', columns='experience', values='skill_count').fillna(0)
# Создаем интерактивный heatmap
fig_2 = px.imshow(
    pivot_df,
    labels=dict(x="Уровень опыта", y="Навык", color="Количество"),
    x=pivot_df.columns,
    y=pivot_df.index,
    color_continuous_scale='YlOrRd',
    aspect="auto",
    title='Топ навыков Data Science по уровням опыта'
)
# Настраиваем отображение
fig_2.update_layout(
    xaxis_title="Уровень опыта",
    yaxis_title="Навыки",
    width=800,
    height=600
)
# Добавляем аннотации с числами
fig_2.update_traces(
    text=pivot_df.values,
    texttemplate="%{text}",
    hovertemplate="<b>%{y}</b><br>Опыт: %{x}<br>Количество: %{z}<extra></extra>"
)

# fig_2.show()
# fig_2.write_html("data/html/fig_2.html")

<img src = 'data/image/fig_2.png'>

Ознакомится, с интерактивным графиком можно по [ссылке](https://htmlpreview.github.io/?https://github.com/emozdir/project_2/blob/master/data/html/fig_2.html)

**Выводы по анализу навыков Data Science:**

+ Python - единственный навык с высокими значениями на всех уровнях, особенно у специалистов с 1-3 годами и 3-6 годами опыта

+ Самые "горячие" (красные) ячейки: Python у специалистов 1-3 года (156) и 3-6 лет (149), SQL у специалистов 1-3 года (124), Machine Learning на всех уровнях
+ Для новичков достаточно базовых знаний математики, Python, SQL и ML
+ Кандидатам senior уровня (более 6 лет) добавилось знание инструментов управления - Atlassian Confluence, Atlassian Jira, Java, ООП. При этом у этого уровня заметно меньше красных ячеек, чем у 1-6 лет опыта. Например, C++ встречается только у специалистов со стажем 3-6 лет (26). Я думаю, это связано с ограниченным количеством вакансий уровня senior.
+ От "нет опыта" к "1-3 года" - резкий рост количества требований, пик востребованности многих навыков приходится на 1-6 лет опыта

Таким образом, наибольшие требования к разнообразным навыкам предъявляются к специалистам с опытом 1-6 лет, а самые senior-разработчики должны дополнительно владеть инструментами управления проектами.

### 6. Динамика зарплат по опыту работы

In [63]:
# Запрашиваем данные по уровню зарплат для каждого опыта, отбрасывая NULL значения
salary_exp_query = f'''
WITH ds_vacancy AS ({ds_vacancy})
SELECT 
    experience,
    COALESCE(
        (salary_from + salary_to)/2,
        salary_from,
        salary_to
        ) AS salary
FROM ds_vacancy
WHERE
    salary_from IS NOT NULL 
    OR salary_to IS NOT NULL
'''
# Получаем DataFrame
salary_exp_df = pd.read_sql_query(salary_exp_query, connection)

In [64]:
# Рисуем боксплот
fig_3 = px.box(
    salary_exp_df,
    x='experience',
    y='salary',
    title='Распределение зарплат Data Science по опыту работы',
    labels={'experience': 'Опыт работы', 'salary': 'Зарплата, руб.'},
    color='experience'  # Раскрашиваем по опыту
)

# Настраиваем оформление
fig_3.update_layout(
    showlegend=False,
    width=800,
    height=600
)

# fig_3.show()
# fig_3.write_html("data/html/fig_3.html")

<img src = 'data/image/fig_3.png'>

Ознакомится, с интерактивным графиком можно по [ссылке](https://htmlpreview.github.io/?https://github.com/emozdir/project_2/blob/master/data/html/fig_3.html)

**Выводы:**

Четкая прогрессия доходов:
+ Нет опыта: 55 000 - 93 000 руб. (узкий диапазон, стабильный уровень)
+ 1-3 года: 95 000 - 175 000 руб. (значительный рост, наибольший разброс)
+ 3-6 лет: 200 000 - 300 000 руб. (высокий базовый уровень, топовые позиции до 450000 руб.)

Ключевые наблюдения:
+ Начинающие: стабильный, но ограниченный доход, минимальные выбросы
+ 1-3 года: максимальный разброс зарплат, что свидетельствует о разных карьерных траекториях и специализациях
+ 3-6 лет: высокий гарантированный доход от 200000 руб., возможность достижения 450000 руб. для топ-специалистов
+ Отсутствуют данные по зарплатам специалистов с опытом более 6 лет

Таким образом, Data Science предлагает одну из самых перспективных карьерных траекторий в IT с возможностью роста дохода в 3-4 раза за 3-6 лет профессионального развития.

## Общие выводы по проекту

**Общий вывод по исследованию рынка вакансий Data Science**

Проведенный анализ выявил, что рынок Data Science представляет собой нишевый сегмент с четко выраженными характеристиками:

**Структура:**
- Data Science занимает всего 1-3.8% от общего IT-рынка, что подтверждает его специализированный характер;
- Ограниченное количество junior-вакансий (51) свидетельствует о высоком пороге входа в профессию;
- Среднее количество навыков на вакансию (6-7) подчеркивает многопрофильность требований.

**Ключевые навыки:**
- **Python и SQL** формируют ядро профессии;
- С ростом опыта добавляются новые навыки (английский язык, Java, ООП, инструменты управления и др.);
- Пик требований к навыкам приходится на специалистов с опытом 1-6 лет.

**Заработные платы и география:**
- Четкая прогрессия доходов: от 74 тыс. руб. у новичков до 243 тыс. руб. у опытных специалистов;
- Москва лидирует по зарплатам среди российских городов, но конкурирует с зарубежными рынками;
- Обнаружены высокие уровни заработной платы в регионах (Белгород, Рязань).

**Перспективы:**
- Возможность удвоения-утроения дохода за 3-6 лет профессионального роста;
- Расширение географии трудоустройства: высокие зарплаты доступны не только в столице;
- Наибольший потенциал роста для специалистов с опытом 1-3 года.

## Перспективы дальнейших исследований и улучшения исходных данных

**В ходе анализа были выявлены следующие проблемы:**
- 35.8% вакансий без указания сферы деятельности компании;
- Нестандартизированные форматы навыков (разный регистр, синонимы);
- Отсутствие унификации в названиях должностей;
- Неполные данные по зарплатам.

**Рекомендации к данным:**
- Внедрение стандартизированных справочников для навыков и специализаций;
- Обязательное указание сферы деятельности компании;
- Обязательное указание зарплатных вилок;
- Сбор данных о времени размещения вакансий.

**Возможные дальнейшие исследования:**

- Тренды изменения зарплат за период;
- Анализ влияния образования на уровень зарплат;
- Исследование вакансий с удаленным типом работы и их оплаты;
- Сравнение российского рынка DS с мировыми тенденциями;
- Исследование влияния размера компании на требования и оплату.

**Заключение:** Совершенствование методологии сбора и обработки данных позволит провести более точные исследования, что особенно важно для цели данного проекта - создания ML модели.

In [ ]:
# Закрываем соединение после окончания работы
connection.close()